In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib.ticker as ticker
from IPython.display import Audio
from sklearn import datasets, linear_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.manifold import TSNE

sns.set()
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 200)

In [2]:
#Features
features = pd.read_excel('Hot 100 Audio Features.xlsx')

In [3]:
#Billboard Top 100 By Week from 1958 to 2021
bb100 = pd.read_csv('Hot Stuff.csv')

In [4]:
#Convert 'WeekID' to DateTime Object & Check First & Last 2 entries by Week/Year
bb100['WeekID'] = pd.DatetimeIndex(bb100['WeekID'])
bb100.sort_values(by='WeekID').iloc[np.r_[0:2, -2:0]]

,url,WeekID,Week Position,Song,Performer,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart
18553,http://www.billboard.com/charts/hot-100/1958-08-02,1958-08-02,63,High School Confidential,Jerry Lee Lewis And His Pumping Piano,High School ConfidentialJerry Lee Lewis And His Pumping Piano,1,NaN,63,1
103337,http://www.billboard.com/charts/hot-100/1958-08-02,1958-08-02,98,Little Serenade,The Ames Brothers,Little SerenadeThe Ames Brothers,1,NaN,98,1
300806,https://www.billboard.com/charts/hot-100/2021-05-29,2021-05-29,61,Almost Maybes,Jordan Davis,Almost MaybesJordan Davis,2,64.0,61,17
152154,https://www.billboard.com/charts/hot-100/2021-05-29,2021-05-29,78,White Teeth,YoungBoy Never Broke Again,White TeethYoungBoy Never Broke Again,1,NaN,78,1


In [5]:
#Merge BB100 with Features
bb100_features = pd.merge(bb100, features, on='SongID', how='left')
print(bb100_features.shape)
bb100_features.head(2)

(330461, 31)


,url,WeekID,Week Position,Song_x,Performer_x,SongID,Instance,Previous Week Position,Peak Position,Weeks on Chart,Performer_y,Song_y,spotify_genre,spotify_track_id,spotify_track_preview_url,spotify_track_duration_ms,spotify_track_explicit,spotify_track_album,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,spotify_track_popularity
0,http://www.billboard.com/charts/hot-100/1965-07-17,1965-07-17,34,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,45.0,34,4,Patty Duke,Don't Just Stand There,['deep adult standards'],1YhNCQ3XOdTCZgubfX8PgB,NaN,163160.0,0.0,Lost Hits Of The 60's (All Original Artists & Versions),0.574,0.256,7.0,-15.044,1.0,0.0298,0.61,0.000077,0.1,0.568,82.331,3.0,21.0
1,http://www.billboard.com/charts/hot-100/1965-07-24,1965-07-24,22,Don't Just Stand There,Patty Duke,Don't Just Stand TherePatty Duke,1,34.0,22,5,Patty Duke,Don't Just Stand There,['deep adult standards'],1YhNCQ3XOdTCZgubfX8PgB,NaN,163160.0,0.0,Lost Hits Of The 60's (All Original Artists & Versions),0.574,0.256,7.0,-15.044,1.0,0.0298,0.61,0.000077,0.1,0.568,82.331,3.0,21.0


In [6]:
#Taking some important columns to introduce Spotify Metrics
subset_1 = bb100_features[['WeekID','Song_x', 'spotify_track_preview_url', 
                           'instrumentalness', 'speechiness',
                           'spotify_track_popularity']]
print(subset_1.shape)
subset_1.head(2)

(330461, 6)


,WeekID,Song_x,spotify_track_preview_url,instrumentalness,speechiness,spotify_track_popularity
0,1965-07-17,Don't Just Stand There,NaN,0.000077,0.0298,21.0
1,1965-07-24,Don't Just Stand There,NaN,0.000077,0.0298,21.0


In [7]:
#Remove all Nan's from subset_1
subset_1 = subset_1.dropna()
print(subset_1.shape)
subset_1.head(2)

(169763, 6)


,WeekID,Song_x,spotify_track_preview_url,instrumentalness,speechiness,spotify_track_popularity
14,1971-04-24,Don't Knock My Love - Pt. 1,https://p.scdn.co/mp3-preview/5d3332b4ae616cd2157f948f7329b20470714a95?cid=b8d3901151d34489a160e3cf0ab1fa94,0.000007,0.0287,26.0
15,1971-05-01,Don't Knock My Love - Pt. 1,https://p.scdn.co/mp3-preview/5d3332b4ae616cd2157f948f7329b20470714a95?cid=b8d3901151d34489a160e3cf0ab1fa94,0.000007,0.0287,26.0


In [8]:
#Remove all duplicate songs from subset_1
subset_1 = subset_1.drop_duplicates(subset='Song_x')
print(subset_1.shape)
subset_1.head(2)

(12689, 6)


,WeekID,Song_x,spotify_track_preview_url,instrumentalness,speechiness,spotify_track_popularity
14,1971-04-24,Don't Knock My Love - Pt. 1,https://p.scdn.co/mp3-preview/5d3332b4ae616cd2157f948f7329b20470714a95?cid=b8d3901151d34489a160e3cf0ab1fa94,0.000007,0.0287,26.0
39,1989-09-30,Don't Know Much,https://p.scdn.co/mp3-preview/49fff23849dc6ec86909dd299908100857831882?cid=b8d3901151d34489a160e3cf0ab1fa94,0.000011,0.0327,60.0


In [48]:
#Sort by Speechiness Score
Speechy = subset_1.sort_values(by='speechiness', ascending=False)
Speechy.head(10)

,WeekID,Song_x,spotify_track_preview_url,instrumentalness,speechiness,spotify_track_popularity
83587,2016-03-05,Think Of You,https://p.scdn.co/mp3-preview/34ed1bd1e231ec802c08ad70653a1c8656830cbb?cid=b8d3901151d34489a160e3cf0ab1fa94,0.000000,0.951,18.0
21849,1972-01-15,Ajax Airlines,https://p.scdn.co/mp3-preview/b83eb127a0ddc4a9939059e5147c024da5096afd?cid=b8d3901151d34489a160e3cf0ab1fa94,0.000000,0.924,5.0
12553,1971-04-24,Ajax Liquor Store,https://p.scdn.co/mp3-preview/c5b1e4e4d1073bcf470accfbceaeba3b5f32f81d?cid=b8d3901151d34489a160e3cf0ab1fa94,0.000000,0.913,8.0
88111,1989-06-17,Trouble Me,https://p.scdn.co/mp3-preview/210672943f13f298ebfea5d4fa06e4048073f475?cid=b8d3901151d34489a160e3cf0ab1fa94,0.000829,0.894,0.0
42157,2020-08-01,The Climb Back,https://p.scdn.co/mp3-preview/251a6ccf0bf7fb59b25f8e32d38cc84a9d60ca46?cid=b8d3901151d34489a160e3cf0ab1fa94,0.000000,0.888,0.0
52268,1977-12-03,Grandmother's Song,https://p.scdn.co/mp3-preview/b699a273cd968947d16c8b7eadcd3b61a1808ab5?cid=b8d3901151d34489a160e3cf0ab1fa94,0.000000,0.858,16.0
14035,1959-02-07,"Blah, Blah, Blah",https://p.scdn.co/mp3-preview/f27300f737288e3a9b8d965ed6fd49aab1c9ae66?cid=b8d3901151d34489a160e3cf0ab1fa94,0.000000,0.855,1.0
43088,1961-04-22,The Touchables In Brooklyn,https://p.scdn.co/mp3-preview/30792d161ad0d9d3aabedeeb50636e388e118ebd?cid=b8d3901151d34489a160e3cf0ab1fa94,0.000033,0.847,0.0
20869,1962-01-13,My Boomerang Won't Come Back,https://p.scdn.co/mp3-preview/a46cfaeda03eb27b7fd7df9e48aaf49a769f35ac?cid=b8d3901151d34489a160e3cf0ab1fa94,0.000000,0.846,15.0
30354,1959-05-16,Russian Band Stand,https://p.scdn.co/mp3-preview/7c04ed76b7b65688df400fdcb92f50d49adb4466?cid=b8d3901151d34489a160e3cf0ab1fa94,0.000000,0.846,0.0


In [49]:
Speechy.describe()

,instrumentalness,speechiness,spotify_track_popularity
count,12689.000000,12689.000000,12689.000000
mean,0.037527,0.071868,39.671369
std,0.149234,0.080615,22.605779
min,0.000000,0.000000,0.000000
25%,0.000000,0.032100,21.000000
50%,0.000005,0.041200,41.000000
75%,0.000573,0.067700,58.000000
max,0.982000,0.951000,100.000000


In [50]:
Speechy = Speechy.loc[(Speechy['speechiness']>0.5) &
                      (Speechy['spotify_track_popularity'] > 10)]

In [51]:
Speechy.shape

(27, 6)

In [52]:
Speechy

,WeekID,Song_x,spotify_track_preview_url,instrumentalness,speechiness,spotify_track_popularity
83587,2016-03-05,Think Of You,https://p.scdn.co/mp3-preview/34ed1bd1e231ec802c08ad70653a1c8656830cbb?cid=b8d3901151d34489a160e3cf0ab1fa94,0.000000,0.951,18.0
52268,1977-12-03,Grandmother's Song,https://p.scdn.co/mp3-preview/b699a273cd968947d16c8b7eadcd3b61a1808ab5?cid=b8d3901151d34489a160e3cf0ab1fa94,0.000000,0.858,16.0
20869,1962-01-13,My Boomerang Won't Come Back,https://p.scdn.co/mp3-preview/a46cfaeda03eb27b7fd7df9e48aaf49a769f35ac?cid=b8d3901151d34489a160e3cf0ab1fa94,0.000000,0.846,15.0
8253,1975-09-06,Mr. Jaws,https://p.scdn.co/mp3-preview/03f5af7f4458f8999d4a228893b7ee926ad35bde?cid=b8d3901151d34489a160e3cf0ab1fa94,0.000000,0.693,12.0
33331,1979-11-24,Cruel Shoes,https://p.scdn.co/mp3-preview/8845a6479ad1e349abd0baee533831bdacd10cf6?cid=b8d3901151d34489a160e3cf0ab1fa94,0.000000,0.693,13.0
125829,2018-12-22,Train Food,https://p.scdn.co/mp3-preview/553342234f9eb0d476174dd4ff908f674609d2de?cid=b8d3901151d34489a160e3cf0ab1fa94,0.000091,0.691,63.0
12842,2020-05-30,Beautiful Pain (Losin My Mind),https://p.scdn.co/mp3-preview/7c99353c8295cc04df2f999d68a5f866a29dfccb?cid=b8d3901151d34489a160e3cf0ab1fa94,0.000000,0.642,71.0
43094,1961-02-18,The Touchables,https://p.scdn.co/mp3-preview/c9357e460633fd4a1af54f0b430ca276d339639b?cid=b8d3901151d34489a160e3cf0ab1fa94,0.000159,0.640,15.0
5406,1959-09-12,Deck Of Cards,https://p.scdn.co/mp3-preview/991734521d5c261ecb280febda76c8fe24931111?cid=b8d3901151d34489a160e3cf0ab1fa94,0.000004,0.618,21.0
4750,1958-08-02,Delicious!,https://p.scdn.co/mp3-preview/93cf9b9d4dbc2da95f0a9ac8f809510e59c5f0c5?cid=b8d3901151d34489a160e3cf0ab1fa94,0.072200,0.584,19.0


In [53]:
#Get 'Got Your Back' preview URL:
Got_Your_Back = Speechy['spotify_track_preview_url'].iloc[26]
Got_Your_Back

'https://p.scdn.co/mp3-preview/9f94e97130e387df884b4af56c6e9fe231983ac9?cid=b8d3901151d34489a160e3cf0ab1fa94'

In [54]:
#Download URL and save as MP3 file: 
import urllib.request
url = Got_Your_Back
filename = 'Speechy_Score_0.501.mp3'
urllib.request.urlretrieve(url, filename)

('Speechy_Score_0.501.mp3', <http.client.HTTPMessage at 0x7f85b1591850>)

In [55]:
#Play MP3 file: 
from IPython.display import Audio
Audio('Speechy_Score_0.501.mp3')